In [1]:
import numpy as np
import os
import pandas as pd
from IPython.display import display

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dspp1/product_info.csv
/kaggle/input/dspp1/customer_product.csv
/kaggle/input/dspp1/customer_info.csv
/kaggle/input/dspp1/customer_cases.csv


#### Load the Dataset

In [3]:
product_info = pd.read_csv("/kaggle/input/dspp1/product_info.csv")
customer_product = pd.read_csv("/kaggle/input/dspp1/customer_product.csv")
customer_info = pd.read_csv("/kaggle/input/dspp1/customer_info.csv")
customer_cases = pd.read_csv("/kaggle/input/dspp1/customer_cases.csv")

# Exploratory Data Analysis

In [4]:
def style_df(df, caption=''):
    return df.style.set_caption(caption).set_table_styles([
        {'selector': 'th', 'props':[('background-color', '#f0f0f0'),
                                   ('color', 'black'),
                                   ('font-weight', 'bold')]},
        {'selector': 'tr:nth-of-type(odd)','props':[('background-color', '#f9f9f9')]},
    ])

In [5]:
style_df(customer_product.head(), 'Customer Sign-up and Cancellation Dates')

,Unnamed: 0,customer_id,product,signup_date_time,cancel_date_time
0,1,C2448,prd_1,2017-01-01 10:35:09,nan
1,2,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02
2,3,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55
3,4,C2451,prd_2,2017-01-01 13:32:08,nan
4,5,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01


In [6]:
style_df(customer_info.head(), 'Customer Demographics')

,Unnamed: 0,customer_id,age,gender
0,1,C2448,76,female
1,2,C2449,61,male
2,3,C2450,58,female
3,4,C2451,62,female
4,5,C2452,71,male


In [7]:
style_df(customer_cases.head(), 'Call Center Activity')

,Unnamed: 0,case_id,date_time,customer_id,channel,reason
0,1,CC101,2017-01-01 10:32:03,C2448,phone,signup
1,2,CC102,2017-01-01 11:35:47,C2449,phone,signup
2,3,CC103,2017-01-01 11:37:09,C2450,phone,signup
3,4,CC104,2017-01-01 13:28:14,C2451,phone,signup
4,5,CC105,2017-01-01 13:52:22,C2452,phone,signup


In [8]:
style_df(product_info.head(), 'Product Information')

,product_id,name,price,billing_cycle
0,prd_1,annual_subscription,1200,12
1,prd_2,monthly_subscription,125,1


Customer Product Table will be the centre of the star schema. customer_id will be the primary key linking to the customer_centre and customer_info. product_id will be the primary key for the product_info table

## Missing Values Check

In [9]:
for df_name, df in [("product_info", product_info), 
                    ("customer_product", customer_product),
                    ("customer_info", customer_info), 
                    ("customer_cases", customer_cases)]:
    print(f"\n{df_name} missing values:")
    print(df.isnull().sum())
    
    print(f"\n{df_name} data types:")
    print(df.dtypes)


product_info missing values:
product_id       0
name             0
price            0
billing_cycle    0
dtype: int64

product_info data types:
product_id       object
name             object
price             int64
billing_cycle     int64
dtype: object

customer_product missing values:
Unnamed: 0               0
customer_id              0
product                  0
signup_date_time         0
cancel_date_time    396447
dtype: int64

customer_product data types:
Unnamed: 0           int64
customer_id         object
product             object
signup_date_time    object
cancel_date_time    object
dtype: object

customer_info missing values:
Unnamed: 0     0
customer_id    0
age            0
gender         0
dtype: int64

customer_info data types:
Unnamed: 0      int64
customer_id    object
age             int64
gender         object
dtype: object

customer_cases missing values:
Unnamed: 0     0
case_id        0
date_time      0
customer_id    0
channel        0
reason         0
dtype: in

In [10]:
customer_product.isnull().sum()

Unnamed: 0               0
customer_id              0
product                  0
signup_date_time         0
cancel_date_time    396447
dtype: int64

This suggests that the missing values in cancel date_time are not actual missing values but rather customers who have not cancelled their subscription

In [11]:
customer_product['signup_date_time'] = pd.to_datetime(customer_product['signup_date_time'])
customer_product['cancel_date_time'] = pd.to_datetime(customer_product['cancel_date_time'])
customer_cases['date_time'] = pd.to_datetime(customer_cases['date_time'])

In [12]:
today = pd.to_datetime('2022-01-01')
customer_product['end_date'] = customer_product['cancel_date_time'].fillna(today)
customer_product['tenure_days'] = (customer_product['end_date'] - customer_product['signup_date_time']).dt.days

## Create a Joined Dataframe

In [13]:
joined_df = pd.merge(customer_product,customer_info, how='left', left_on='customer_id', right_on='customer_id')

In [14]:
joined_df = pd.merge(joined_df,product_info, how='left', left_on='product',right_on='product_id')

In [15]:
case_summary = customer_cases.groupby('customer_id').agg({
	'case_id': 'count',
	'date_time':['min','max']
})

In [16]:
case_summary.columns = [
	'total_cases',
	'first_case_date',
	'last_case_date'
]

In [17]:
case_summary['days_between_cases'] = (case_summary['last_case_date'] - case_summary['first_case_date']).dt.days

In [18]:
reason_counts = customer_cases.groupby(['customer_id', 'reason']).size().unstack(fill_value=0)
reason_counts.columns = [f"reason{col}" for col in reason_counts.columns]
case_summary = case_summary.join(reason_counts)

In [19]:
churn_df = pd.merge(joined_df, case_summary, on='customer_id', how='left')

In [20]:
churn_df['total_cases'] = churn_df['total_cases'].fillna(0)

In [21]:
churn_df = churn_df.drop('Unnamed: 0_x', axis=1)
churn_df = churn_df.drop('Unnamed: 0_y', axis=1)

In [22]:
print(churn_df.isnull().sum())

customer_id                0
product                    0
signup_date_time           0
cancel_date_time      396447
end_date                   0
tenure_days                0
age                        0
gender                     0
product_id                 0
name                       0
price                      0
billing_cycle              0
total_cases                0
first_case_date       250272
last_case_date        250272
days_between_cases    250272
reasonsignup          250272
reasonsupport         250272
dtype: int64


## Clean Merged Dataset

In [23]:
churn_df['total_cases'] = churn_df['total_cases'].fillna(0)
churn_df['first_case_date'] = churn_df['first_case_date'].fillna(pd.NaT)
churn_df['last_case_date'] = churn_df['last_case_date'].fillna(pd.NaT)
churn_df['days_between_cases'] = churn_df['days_between_cases'].fillna(0)
churn_df['ever_contacted_support'] = churn_df['total_cases'].gt(0).astype(int)

In [24]:
reason_columns = [col for col in churn_df.columns if col.startswith('reason')]
churn_df[reason_columns] = churn_df[reason_columns].fillna(0)

In [25]:
print(churn_df.isnull().sum())

customer_id                    0
product                        0
signup_date_time               0
cancel_date_time          396447
end_date                       0
tenure_days                    0
age                            0
gender                         0
product_id                     0
name                           0
price                          0
billing_cycle                  0
total_cases                    0
first_case_date           250272
last_case_date            250272
days_between_cases             0
reasonsignup                   0
reasonsupport                  0
ever_contacted_support         0
dtype: int64


In [26]:
churn_df['days_since_signup'] = (today - churn_df['signup_date_time']).dt.days 
mask = churn_df['ever_contacted_support'] == 1
churn_df.loc[mask,'days_to_first_contact'] = (churn_df.loc[mask, 'first_case_date'] - churn_df.loc[mask, 'signup_date_time']).dt.days

In [27]:
churn_df['days_to_first_contact'] = churn_df['days_to_first_contact'].fillna(-1)

In [28]:
churn_df['age_group'] = pd.cut(churn_df['age'],
                              bins  = [0, 30, 45, 60, 75, 100],
                              labels  = ['<30', '30-45', '46-60','61-75','75+'])

In [29]:
churn_df['tenure_group'] = pd.cut(churn_df['tenure_days'],
                                 bins = [0, 90, 180, 365, 730, float('inf')],
                                 labels = ['0-3mo', '3-6mo', '6-12mo','1-2yr', '2yr+'])

In [30]:
churn_df.head()

,customer_id,product,signup_date_time,cancel_date_time,end_date,tenure_days,age,gender,product_id,name,...,first_case_date,last_case_date,days_between_cases,reasonsignup,reasonsupport,ever_contacted_support,days_since_signup,days_to_first_contact,age_group,tenure_group
0,C2448,prd_1,2017-01-01 10:35:09,NaT,2022-01-01 00:00:00,1825,76,female,prd_1,annual_subscription,...,2017-01-01 10:32:03,2017-01-01 10:32:03,0.0,1.0,0.0,1,1825,-1.0,75+,2yr+
1,C2449,prd_1,2017-01-01 11:39:29,2021-09-05 10:00:02,2021-09-05 10:00:02,1707,61,male,prd_1,annual_subscription,...,2017-01-01 11:35:47,2017-01-01 11:35:47,0.0,1.0,0.0,1,1825,-1.0,61-75,2yr+
2,C2450,prd_1,2017-01-01 11:42:00,2019-01-13 16:24:55,2019-01-13 16:24:55,742,58,female,prd_1,annual_subscription,...,2017-01-01 11:37:09,2017-01-01 11:37:09,0.0,1.0,0.0,1,1825,-1.0,46-60,2yr+
3,C2451,prd_2,2017-01-01 13:32:08,NaT,2022-01-01 00:00:00,1825,62,female,prd_2,monthly_subscription,...,2017-01-01 13:28:14,2017-03-31 12:06:58,88.0,1.0,1.0,1,1825,-1.0,61-75,2yr+
4,C2452,prd_1,2017-01-01 13:57:30,2021-06-28 18:06:01,2021-06-28 18:06:01,1639,71,male,prd_1,annual_subscription,...,2017-01-01 13:52:22,2017-01-01 13:52:22,0.0,1.0,0.0,1,1825,-1.0,61-75,2yr+
